In [1]:
# 原始模型

import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from tqdm import tqdm,trange
import torch.nn.functional as F
import numpy as np 
from Datasets import AngleActionDatasets

In [3]:
classes = 5  #分类
hidden_dim = 64 # rnn隐藏单元数
lr = 0.001 # 学习率
epoches = 20 #训练次数
batch_size = 1024 # 每一个训练批次数量
input_dim= 7 * 3 
device = "cuda" if torch.cuda.is_available() else "cpu"
time_step = 40
print(device)

cuda


In [4]:
def create_data_loader():
    train_data_path = "D:\\temp\\attitudeAngle_windows"
    # train_data_path = "D:\\temp\\augment_action_windows"
    datasets = AngleActionDatasets(train_data_path, transform=torch.tensor, target_transform=torch.tensor, pick_path='train.pkl')
    test_datasets = AngleActionDatasets("D:\\temp\\yan1_action_windows\\attitudeAngle", transform=torch.tensor, target_transform=torch.tensor, pick_path='test.pkl')
    split_rate = 0.8  # 训练集占整个数据集的比例
    train_len = int(split_rate * len(datasets))
    valid_len = len(datasets) - train_len

    train_sets, valid_sets = random_split(datasets, [train_len, valid_len], generator=torch.Generator().manual_seed(42))

    train_loader = DataLoader(train_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)
    test_loader = DataLoader(test_datasets, shuffle=True,pin_memory=True)
    valid_loader = DataLoader(valid_sets, batch_size=batch_size, shuffle=True,drop_last=True,pin_memory=True)

    print(f"训练集大小{len(train_sets)}， 验证集大小{len(valid_sets)}")
    return train_loader, test_loader, valid_loader
train_loader, test_loader, valid_loader = create_data_loader()

训练集大小63980， 验证集大小15996


In [5]:
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, out_dim, time_step):
        super(RNN, self).__init__()
        self.time_step = time_step
        self.hidden_dim2 = hidden_dim2
        self.hidden_dim1 = hidden_dim1
        self.linear1 = nn.Linear(input_dim, hidden_dim1)
        self.linear2 = nn.Linear(input_dim, hidden_dim1)
        self.linear3 = nn.Linear(hidden_dim2, time_step)
        self.linear4 = nn.Linear(hidden_dim2, out_dim)

        self.rnn1 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.rnn2 = nn.LSTM(hidden_dim2, hidden_dim2, batch_first=True)
        self.dp1 = nn.Dropout(p=0.3)
        self.dp2 = nn.Dropout(p=0.3)
        self.dp3 = nn.Dropout(p=0.6)
        
    def forward(self, X):
        # X.shape = batch_size, time_step, feature_num
        A, B = X
        batch_size = A.shape[0]
        # out.shape = (batch_size, time_step, hidden_dim1)
        out1 = self.dp1(F.relu(self.linear1(B)))
        out2 = self.dp2(F.relu(self.linear2(A)))

        # out1.shape = out.shape,  status1 = (h, c)
        # h.shape = c.shape = (方向* 层数, batch_size, hidden_dim2)
        out1, (h1, c1) = self.rnn1(out1)

        h1 = h1.view(batch_size, self.hidden_dim2)
        out11 = self.dp3(F.relu(self.linear3(h1)))

        out11 = out11.unsqueeze(2).expand(batch_size, self.time_step, self.hidden_dim1)

        out3 = out2 + out11
        h2 = out3.mean(dim=1).unsqueeze(0)
        out, _ = self.rnn2(out1, (h2, torch.zeros_like(h2)))

        out = self.linear4(out[:,-1,:])

        return out

In [6]:
rnn = RNN(input_dim, hidden_dim, hidden_dim, classes, time_step).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=lr)

In [7]:
def GETACC(loader=valid_loader):
    rnn.eval()
    cnt = 0
    sum_valid_acc = 0
    sum_valid_loss = 0
    for data, label in loader:
        data = [item.to(device) for item in data]
        label = label.to(device)
        out = rnn(data)
        
        _, predict = torch.max(out, 1)
        
        loss = criterion(out, label)
        sum_valid_loss += loss.item()
        acc = torch.mean((predict == label).float())
        sum_valid_acc += acc
        cnt+=1
        
    return sum_valid_loss/cnt, sum_valid_acc/cnt

In [8]:
for epoch in range(epoches):
    i = 0
    loss_sum = 0
    bar = tqdm(train_loader)
    for ii, (data , label) in enumerate(bar):
        rnn.train()
    
        data = [item.to(device) for item in data]
        label = label.to(device)
        out = rnn(data)
        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        i+=1
        loss_sum += loss.item()

        if ii == 0:
            valid_loss,valid_acc = GETACC(valid_loader)
            
            bar.set_description(f"epoch = {epoch} train_loss = {loss_sum/i} valid_loss = {valid_loss} valid_acc= {valid_acc}")


epoch = 0 train_loss = 1.6126844882965088 valid_loss = 1.607137934366862 valid_acc= 0.19993489980697632: 100%|██████████| 62/62 [00:07<00:00,  8.53it/s]
epoch = 1 train_loss = 0.5170836448669434 valid_loss = 0.551896337668101 valid_acc= 0.8130208849906921: 100%|██████████| 62/62 [00:06<00:00,  8.93it/s]
epoch = 2 train_loss = 0.3488447368144989 valid_loss = 0.3491211990515391 valid_acc= 0.8777344226837158: 100%|██████████| 62/62 [00:06<00:00,  9.66it/s]
epoch = 3 train_loss = 0.2600637674331665 valid_loss = 0.27132836282253264 valid_acc= 0.90130215883255: 100%|██████████| 62/62 [00:06<00:00,  9.74it/s]
epoch = 4 train_loss = 0.2478613257408142 valid_loss = 0.22089275519053142 valid_acc= 0.9191406965255737: 100%|██████████| 62/62 [00:06<00:00,  9.97it/s]
epoch = 5 train_loss = 0.17582334578037262 valid_loss = 0.19349854389826457 valid_acc= 0.9263021349906921: 100%|██████████| 62/62 [00:06<00:00, 10.26it/s]
epoch = 6 train_loss = 0.16198143362998962 valid_loss = 0.17846474051475525 valid

In [9]:
test_loss,test_acc = GETACC(test_loader)
print(f"test_loss = {test_loss}, test_acc = {test_acc}")

test_loss = 1.0351570993453665, test_acc = 0.7542288899421692
